# Import libraries

In [1]:
import requests
import warnings
import pandas as pd
import time
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime

# Data Scrapping

In [33]:
# Define the season
season = "2017-2018"  

# Construct the dynamic URL
standings_url = f"https://fbref.com/en/comps/9/{season}/{season}-Premier-League-Stats"
data = requests.get(standings_url)

soup = BeautifulSoup(data.text)

In [34]:
#Find all <table> elements with class "stats_table"
PL_table = soup.select('table.stats_table')[0]

#Finds all <a> elements in the Premier League table
links = PL_table.find_all('a')

#Extract only the href attribute from each <a> tag and store all links as a list
links = [l.get("href") for l in links]

#Keep only links containing /squads/
links = [l for l in links if '/squads/' in l]

#Reconstruct full URLs
squad_urls = [f"https://fbref.com{l}" for l in links]

squad_urls

['https://fbref.com/en/squads/b8fd03ef/2017-2018/Manchester-City-Stats',
 'https://fbref.com/en/squads/19538871/2017-2018/Manchester-United-Stats',
 'https://fbref.com/en/squads/361ca564/2017-2018/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/822bd0ba/2017-2018/Liverpool-Stats',
 'https://fbref.com/en/squads/cff3d9bb/2017-2018/Chelsea-Stats',
 'https://fbref.com/en/squads/18bb7c10/2017-2018/Arsenal-Stats',
 'https://fbref.com/en/squads/943e8050/2017-2018/Burnley-Stats',
 'https://fbref.com/en/squads/d3fd31cc/2017-2018/Everton-Stats',
 'https://fbref.com/en/squads/a2d435b3/2017-2018/Leicester-City-Stats',
 'https://fbref.com/en/squads/b2b47a98/2017-2018/Newcastle-United-Stats',
 'https://fbref.com/en/squads/47c64c55/2017-2018/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/4ba7cbea/2017-2018/Bournemouth-Stats',
 'https://fbref.com/en/squads/7c21e445/2017-2018/West-Ham-United-Stats',
 'https://fbref.com/en/squads/2abfe087/2017-2018/Watford-Stats',
 'https://fbref.com/en/sq

In [35]:
warnings.simplefilter(action='ignore', category=FutureWarning)

start_time = datetime.now()
print(f"Start time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

all_teams_data = []
loop_count = 0

for squad_url in squad_urls:
    loop_count += 1
    team_name = squad_url.split("/")[-1].replace("-Stats","").replace("-"," ")

    #Scores & Fixtures
    data = requests.get(squad_url)
    matches = pd.read_html(data.text, match ="Scores & Fixtures")
    soup = BeautifulSoup(data.text)
    shooting_links = soup.find_all('a')
    shooting_links = [l.get("href") for l in shooting_links]
    shooting_links = [l for l in shooting_links if l and 'all_comps/shooting/' in l]
    shooting_page = requests.get(f"https://fbref.com{shooting_links[0]}")
    shooting_stats = pd.read_html(shooting_page.text, match ="Shooting")[0]
    shooting_stats.columns = shooting_stats.columns.droplevel()
    team_data = matches[0].drop(columns=["Match Report", "Notes"])
    team_data = team_data.merge(shooting_stats[["Date","Sh","SoT","SoT%","G/Sh","G/SoT","Dist","FK","PK","PKatt","npxG","npxG/Sh","G-xG","np:G-xG"]], on="Date")
    time.sleep(7)

    #Goalkeeping
    soup = BeautifulSoup(data.text)
    goalkeeping_links = soup.find_all('a')
    goalkeeping_links = [l.get("href") for l in goalkeeping_links]
    goalkeeping_links = [l for l in goalkeeping_links if l and 'all_comps/keeper/' in l]
    goalkeeping_page = requests.get(f"https://fbref.com{goalkeeping_links[0]}")
    goalkeeping_stats = pd.read_html(goalkeeping_page.text, match ="Goalkeeping")[0]
    goalkeeping_stats.columns = goalkeeping_stats.columns.droplevel()
    team_data = team_data.merge(goalkeeping_stats[["Date","SoTA","Saves","Save%","CS"]], on="Date")
    time.sleep(7)

    #Passing
    soup = BeautifulSoup(data.text)
    passing_links = soup.find_all('a')
    passing_links = [l.get("href") for l in passing_links]
    passing_links = [l for l in passing_links if l and 'all_comps/passing/' in l]
    passing_page = requests.get(f"https://fbref.com{passing_links[0]}")
    passing_stats = pd.read_html(passing_page.text, match ="Passing")[0]
    passing_stats.columns = passing_stats.columns.droplevel()
    passing_stats.columns.values[10:24] = ["Total_Cmp", "Total_Att", "Total_Cmp%", "Total_TotDist", "Total_PrgDist",
                                           "Short Pass Cmp","Short Pass Att","Short Pass Cmp%","Medium Pass Cmp","Medium Pass Att","Medium Pass Cmp%",
                                           "Long Pass Cmp","Long Pass Att","Long Pass Cmp%"]
    passing_merge_cols = passing_stats.iloc[:, np.r_[0, 10:32]]
    team_data = team_data.merge(passing_merge_cols, on="Date", how="left")
    time.sleep(7)

    #Pass Types
    soup = BeautifulSoup(data.text)
    pass_types_links = soup.find_all('a')
    pass_types_links = [l.get("href") for l in pass_types_links]
    pass_types_links = [l for l in pass_types_links if l and 'all_comps/passing_types/' in l]
    pass_types_page = requests.get(f"https://fbref.com{pass_types_links[0]}")
    pass_types_stats = pd.read_html(pass_types_page.text, match ="Pass Types")[0]
    pass_types_stats.columns = pass_types_stats.columns.droplevel()
    pass_types_stats.columns.values[10:25] = ["Passes Attempted","Live-ball Passes", "Dead-ball Passes", "Passes from Free Kicks", "Through Balls", "Switches",
                                           "Crosses","Throw-ins Taken","Corner Kicks","Inswinging Corner Kick","Outswinging Corner Kicks","Straight Corner Kicks",
                                           "Passes Completed","Passes Offside","Passes Blocked"]   
    pass_types_merge_cols = pass_types_stats.iloc[:, np.r_[0, 10:25]]
    team_data = team_data.merge(pass_types_merge_cols, on="Date", how="left")
    time.sleep(7)

    #Goal and Shot Creation
    soup = BeautifulSoup(data.text)
    gsc_links = soup.find_all('a')
    gsc_links = [l.get("href") for l in gsc_links]
    gsc_links = [l for l in gsc_links if l and 'all_comps/gca/' in l]
    gsc_page = requests.get(f"https://fbref.com{gsc_links[0]}")
    gsc_stats = pd.read_html(gsc_page.text, match ="Goal and Shot Creation")[0]
    gsc_stats.columns = gsc_stats.columns.droplevel()
    gsc_stats.columns.values[11:24] = ["SCA (Live-ball Pass)","SCA (Dead-ball Pass)", "SCA (Take-On)", "SCA (Shot)", "SCA (Fouls Drawn)", "SCA (Defensive Action)",
                                           "GCA","GCA (Live-ball Pass)","GCA (Dead-ball Pass)","GCA (Take-On)","GCA (Shot)","GCA (Fouls Drawn)",
                                           "GCA (Defensive Action)"]
    gsc_merge_cols = gsc_stats.iloc[:, np.r_[0, 10:24]]
    team_data = team_data.merge(gsc_merge_cols, on="Date", how="left")
    time.sleep(7)

    #Defensive Actions
    soup = BeautifulSoup(data.text)
    dfa_links = soup.find_all('a')
    dfa_links = [l.get("href") for l in dfa_links]
    dfa_links = [l for l in dfa_links if l and 'all_comps/defense/' in l]
    dfa_page = requests.get(f"https://fbref.com{dfa_links[0]}")
    dfa_stats = pd.read_html(dfa_page.text, match ="Defensive Actions")[0]
    dfa_stats.columns = dfa_stats.columns.droplevel()
    dfa_stats.columns.values[10:26] = ["Tackles","Tackles Won", "Tackles (Def 3rd)", "Tackles (Mid 3rd)", "Tackles (Att 3rd)", "Dribblers Tackled",
                                           "Dribbles Challenged","% of Dribblers Tackled","Challenges Lost","Blocks ","Shots Blocked","Passes Blocked",
                                           "Interceptions","Tkl+Int","Clearances","Errors"]
    dfa_merge_cols = dfa_stats.iloc[:, np.r_[0, 10:26]]
    team_data = team_data.merge(dfa_merge_cols, on="Date", how="left")
    time.sleep(7)

    #Possession
    soup = BeautifulSoup(data.text)
    possession_links = soup.find_all('a')
    possession_links = [l.get("href") for l in possession_links]
    possession_links = [l for l in possession_links if l and 'all_comps/possession/' in l]
    possession_page = requests.get(f"https://fbref.com{possession_links[0]}")
    possession_stats = pd.read_html(possession_page.text, match ="Possession")[0]
    possession_stats.columns = possession_stats.columns.droplevel()
    
    possession_stats.columns.values[10:33] = ["Possession %","Touches", "Touches (Def Pen)", "Touches (Def 3rd)", "Touches (Mid 3rd)", "Touches (Att 3rd)",
                                              "Touches (Att Pen)","Touches (Live-Ball)","Take-Ons Attempted","Successful Take-Ons ","Successful Take-On %",
                                              "Times Tackled During Take-On","Tackled During Take-On Percentage","Carries ","Total Carrying Distance",
                                              "Progressive Carrying Distance","Progressive Carries","Carries into Final Third","Carries into Penalty Area",
                                              "Miscontrols","Dispossessed","Passes Received","Progressive Passes Rec"]
    possession_merge_cols = possession_stats.iloc[:, np.r_[0, 10:33]]
    team_data = team_data.merge(possession_merge_cols, on="Date", how="left")
    time.sleep(7)

    #Only keep Premier League games
    team_data = team_data[team_data["Comp"] == "Premier League"]

    #Add season and team colummn data
    team_data["Season"] = season
    team_data["Team"] = team_name

    all_teams_data.append(team_data)
    print(f"{loop_count}. {team_name} complete")

all_teams_data_df = pd.concat(all_teams_data, ignore_index=True)

end_time = datetime.now()
print(f"End time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")

# Calculate execution time
execution_time = end_time - start_time
print(f"Execution Time: {execution_time}")

csv_filename = f"{season}.csv"
all_teams_data_df.to_csv(csv_filename, index=False, encoding="utf-8")

Start time: 2025-01-31 11:38:45
1. Manchester City complete
2. Manchester United complete
3. Tottenham Hotspur complete
4. Liverpool complete
5. Chelsea complete
6. Arsenal complete
7. Burnley complete
8. Everton complete
9. Leicester City complete
10. Newcastle United complete
11. Crystal Palace complete
12. Bournemouth complete
13. West Ham United complete
14. Watford complete
15. Brighton and Hove Albion complete
16. Huddersfield Town complete
17. Southampton complete
18. Swansea City complete
19. Stoke City complete
20. West Bromwich Albion complete
End time: 2025-01-31 12:01:14
Execution Time: 0:22:28.991431


# Load Data Scrapped

In [249]:
df_2017 = pd.read_csv("2017-2018.csv")
df_2018 = pd.read_csv("2018-2019.csv")
df_2019 = pd.read_csv("2019-2020.csv")
df_2020 = pd.read_csv("2020-2021.csv")
df_2021 = pd.read_csv("2021-2022.csv")
df_2022 = pd.read_csv("2022-2023.csv")
df_2023 = pd.read_csv("2023-2024.csv")
df_2024 = pd.read_csv("2024-2025.csv")

In [250]:
combined_df = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023, df_2024], ignore_index=True)

In [294]:
combined_df.to_csv("2017-2024_match_data.csv", index=False)